In [8]:
import numpy as np
import xtrack as xt
import xobjects as xo
import xpart as xp
import numpy as np
import matplotlib.pyplot as plt

In [9]:
#lattice parameters for ELENA at 100 keV
# from https://acc-models.web.cern.ch/acc-models/elena/scenarios/highenergy/highenergy.tfs
qx = 2.36168984503
qy = 1.38992572490
circumference = 30.40531277976 #m

# relativistic factors
gamma_rel = 1.0001066 # at 100 keV

# optics at e-cooler (approximate), in m
beta_x = 1.7
beta_y = 2.7
D_x = 1

# electron cooler parameters
current = 0.34*1e-3 # A current
length = 1 # m cooler length
radius_e_beam = 14*1e-3 #m radius of the electron beam
temp_perp = 100e-3 # <E> [eV] = kb*T
temp_long =  1e-3 # <E> [eV]
magnetic_field = 0.010 # 100 Gauss in ELENA
# idea is to study magnetic field imperfections
magnetic_field_ratio_list = [0,5e-4,1e-3,5e-3] #Iterate over different values of the magnetic field quality to see effect on cooling performance.
#magnetic_field_ratio is the ratio of transverse componenet of magnetic field and the longitudinal component. In the ideal case, the ratio is 0.

# simulation parameters: simulate 10 s of cooling, and take data once every 10 ms
max_time_s = 10
int_time_s = 0.01

qs=0.007718714437902285
bets0=-469.32883416451523

arc_matching = xt.LineSegmentMap(
        qx=qx, qy=qy,
        dqx=0, dqy=0,
        length=circumference,
        betx=beta_x,
        bety=beta_y,
        dx=D_x,
        qs=qs,
        bets=bets0)

line_matching=xt.Line([arc_matching])
line_matching.build_tracker()

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [10]:
# some constants, and simple computations
clight = 299792458.0
mass0 = 938.27208816*1e6 #ev/c^2

beta_rel = np.sqrt(gamma_rel**2 - 1)/gamma_rel
p0c = mass0*beta_rel*gamma_rel #eV/c
T_per_turn = circumference/(clight*beta_rel)

particle_ref= xp.Particles(mass0=mass0, q0=1, p0c=p0c)


line_matching.particle_ref = particle_ref

beta_gamma = line_matching.particle_ref._beta0*line_matching.particle_ref._gamma0

n_part = int(2e4)

beta_x_sem = 7.6 #m
beta_y_sem = 1.3 #m

# create desired beam
bunch_intensity = None
sigma_dp = 1e-3
sigma_z=-bets0*sigma_dp

eps_x= 2.25*1e-6-((D_x*sigma_dp)**2/beta_x_sem)
gemitt_x = 2.5*1e-6
gemitt_y = 2.5*1e-6
nemitt_x = gemitt_x*beta_gamma
nemitt_y = gemitt_y*beta_gamma

particles_old = xp.generate_matched_gaussian_bunch(
        num_particles=n_part,
        # total_intensity_particles=bunch_intensity,
        nemitt_x=nemitt_x, nemitt_y=nemitt_y, sigma_z = 0.3284, # in m,
        particle_ref=particle_ref ,
        line=line_matching,        
)

# particles_old.delta = np.random.normal(0, sigma_dp, n_part)
particles_old.zeta = np.random.uniform(-circumference/2, circumference/2, n_part)

In [11]:
from tqdm import tqdm


data=np.load('V_angles/angles.npz')

n_steps=data['n_steps']
num_samples=data['num_samples']
delay_list=data['delay_list']
repeated_delay=data['repeated_delay']
angles_list=data['angles_list']

angles_list=[-3,-2,-1,0,1,2,3]

sorted_unique_values = np.sort(np.unique(repeated_delay))
second_min_nonzero_value = sorted_unique_values[1]
simulation_time=(np.max(repeated_delay)-second_min_nonzero_value)/1000

# angle_list=np.linspace(-40e-3,40e-3,50)

final_emittance_x=[]
final_emittance_x_normalised=[]
sigma_x_list=[]

final_emittance_y=[]
final_emittance_y_normalised=[]
sigma_y_list=[]

# simulation parameters: simulate 10 s of cooling, and take data once every 10 ms
max_time_s = simulation_time
int_time_s = 0.01

# compute length of simulation, as well as sample interval, in turns
num_turns = int(max_time_s/T_per_turn)
save_interval = int(int_time_s/T_per_turn)


#plot some overall values

arc = xt.LineSegmentMap(
                qx=qx, qy=qx,
                dqx=0, dqy=0,
                length=circumference,
                betx=beta_x,
                bety=beta_y,
                dx=D_x)



for angle in tqdm(angles_list):

        data = np.load(f'results/angle_y/horizontal/ELENA_angle{angle}.npz')

        # Extract the data using the keys used when saving
        h_delay_unique = data['h_delay_unique']
        h_delay_unique_shifted = h_delay_unique[1:]
        h_delay_unique_shifted=h_delay_unique_shifted-h_delay_unique_shifted[0]

        means_h = data['means_h']
        initial_horizontal_emittance = means_h[1] *1e-6 # because first point was faulty

        data = np.load(f'results/angle_y/vertical/ELENA_angle{angle}.npz')
        means_v = data['means_v']        
        initial_vertical_emittance = means_v[1] *1e-6 # because first point was faulty
        # Define the whole machine
        
        electron_cooler = xt.ElectronCooler(
                length=length,
                radius_e_beam=radius_e_beam,
                current=current,
                temp_perp=temp_perp,
                temp_long=temp_long,
                magnetic_field=magnetic_field, 
                magnetic_field_ratio=1e-3,
                offset_py=angle*1e-3,
                space_charge=1)

        # create a monitor object, to reduce holded data
        monitor = xt.ParticlesMonitor(start_at_turn=0, stop_at_turn=1,
                                n_repetitions=int(num_turns/save_interval),
                                repetition_period=save_interval,
                                num_particles=n_part)

        line = xt.Line(
                elements=[monitor, electron_cooler, arc])
        line.particle_ref = xp.Particles(mass0=mass0, q0=1, p0c=p0c)
        context = xo.ContextCpu(omp_num_threads=4)
        line.build_tracker(_context=context)

        particles=particles_old.copy()

        # just track all particles, and keep turn-by-turn data (memory expensive!)
        line.track(particles, num_turns=num_turns,
                turn_by_turn_monitor=False)

        # extract relevant values
        x = monitor.x[:,:,0]
        px = monitor.px[:,:,0]
        y = monitor.y[:,:,0]
        py = monitor.py[:,:,0]
        delta = monitor.delta[:,:,0]
        zeta = monitor.zeta[:,:,0]
        time = monitor.at_turn[:, 0, 0] * T_per_turn

        # compute actions. for x, remove the dp/p contribution:
        action_x = ((x-D_x*delta)**2/beta_x + beta_x*px**2)
        # for y, simple compute:
        action_y = (y**2/beta_y + beta_y*py**2)

        norm_emittance_x=np.mean(action_x,axis=1)/2*gamma_rel*beta_rel
        norm_emittance_y=np.mean(action_y,axis=1)/2*gamma_rel*beta_rel

        geo_emittance_x=np.mean(action_x,axis=1)/2
        geo_emittance_y=np.mean(action_y,axis=1)/2


        np.savez(f'results/angle_y/angle_{angle}.npz',
                
                # emittance_x_twiss=emittance_x_twiss,
                # emittance_y_twiss=emittance_y_twiss,
                geo_emittance_x=geo_emittance_x,
                geo_emittance_y=geo_emittance_y,
                norm_emittance_x=norm_emittance_x,
                norm_emittance_y=norm_emittance_y,
                x=x,  
                y=y,  
                px=px,  
                py=py,
                delta=delta,
                zeta=zeta,
                time=time)  

  0%|          | 0/7 [00:00<?, ?it/s]

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


 14%|█▍        | 1/7 [04:17<25:46, 257.76s/it]

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


 29%|██▊       | 2/7 [08:32<21:20, 256.04s/it]

...

 71%|███████▏  | 5/7 [20:35<08:07, 243.95s/it]

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


 86%|████████▌ | 6/7 [24:35<04:02, 242.74s/it]

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


100%|██████████| 7/7 [28:40<00:00, 245.72s/it]


In [12]:
beta_gamma

array([0.01460176])

In [13]:
geo_emittance_x[0]

2.4739628582582312e-06